## Task 2
### Solving linear system by iterative method


In [1]:
from typing import Literal, Tuple
import numpy as np

In [10]:
def convert_linear_to_iterative_form(
    matrix: np.ndarray,
    right_part: np.ndarray,
    matrix_type: Literal['diagonally_dominant', 'definite']
) -> Tuple[np.ndarray, np.ndarray]:
    H = np.zeros(matrix.shape)
    g = np.zeros(matrix.shape[0])

    if (matrix_type == 'diagonally_dominant'):
        for i in range(matrix.shape[0]):
            g[i] = right_part[i] / matrix[i, i]
            for j in range(matrix.shape[0]):
                if i == j:
                    H[i, j] = 0
                else:
                    H[i, j] = -matrix[i, j] / matrix[i, i]
    if (matrix_type == 'definite'):
        
    return H, g
    print(H)
    print(g)


convert_linear_to_iterative_form(
    np.array([[1, 2], [1, 2]], np.int32), np.array([1, 2]), 'diagonally_dominant')


[[ 0.  -2. ]
 [-0.5  0. ]]
[1. 1.]
